In [1]:
from kubernetes import client, config,utils
import pprint

# Kubernetes Api Client


## Cluster config & Client 정의

In [2]:
config.load_kube_config()
k8s_client = client.ApiClient() 

# Kubernetes CRD Client

In [3]:
config.load_kube_config()
api = client.CustomObjectsApi()

# CRD Get

In [4]:
resource = api.get_namespaced_custom_object(
    group="scheduling.volcano.sh",
    version="v1beta1",
    name="podgroup-e029423c-dc67-45b9-9f80-ebd941fc1605",
    namespace="argo",
    plural="podgroups"
)

In [5]:
import pprint 
pprint.pprint(resource)

{'apiVersion': 'scheduling.volcano.sh/v1beta1',
 'kind': 'PodGroup',
 'metadata': {'creationTimestamp': '2023-05-30T10:56:45Z',
              'generation': 2609,
              'managedFields': [{'apiVersion': 'scheduling.volcano.sh/v1beta1',
                                 'fieldsType': 'FieldsV1',
                                 'fieldsV1': {'f:metadata': {'f:ownerReferences': {'.': {},
                                                                                   'k:{"uid":"e029423c-dc67-45b9-9f80-ebd941fc1605"}': {'.': {},
                                                                                                                                        'f:apiVersion': {},
                                                                                                                                        'f:blockOwnerDeletion': {},
                                                                                                                                        'f:con

# CRD Information

In [7]:
status = resource['status']['phase'] # 상태
priorityClassName = resource['spec']['priorityClassName'] # penalty value
queue = resource['spec']['queue'] # queue

print(status,priorityClassName,queue)

Completed h-priority default


# Volcano 스케줄러 사이클

1. Kubeflow에서 pipeline을 돌림  
2. Argo workflow 형태로 작업이 시작됨  
    - workflow 형태는 시작되면 Active 완료시 Succeeded 상태가 됨  
3. schedulerName과 podPriorityClassName, queue 정보 등을 기입함  
4. 생성된 workflow는 지정한 queue에 podgroup의 crd 형태로 시작된다.  
5. 완료된 podgroup은 삭제되지 않는다.
    - podgroup은 namespace, queue별로 지정되서 들어간다.
    - 이걸 활용해서 penalty를 계산해야하나?  
6. 패널티 계산 로직에 따라 패널티를 계산하고 완화한다.
    - 패널티는 priorityclass의 value를 통해 계산한다.

# PriorityClassName GET

In [8]:
priorityClassResource = api.get_cluster_custom_object(
    group="scheduling.k8s.io",
    version="v1",
    name="h-priority",
    plural="priorityclasses"
)

In [9]:
pprint.pprint(priorityClassResource)

{'apiVersion': 'scheduling.k8s.io/v1',
 'kind': 'PriorityClass',
 'metadata': {'creationTimestamp': '2023-05-26T06:44:33Z',
              'generation': 1,
              'managedFields': [{'apiVersion': 'scheduling.k8s.io/v1',
                                 'fieldsType': 'FieldsV1',
                                 'fieldsV1': {'f:preemptionPolicy': {},
                                              'f:value': {}},
                                 'manager': 'OpenAPI-Generator',
                                 'operation': 'Update',
                                 'time': '2023-05-26T06:44:33Z'}],
              'name': 'h-priority',
              'resourceVersion': '15476',
              'uid': 'bfbe9c75-151e-43e7-90b8-2d23fca8ce9a'},
 'preemptionPolicy': 'PreemptLowerPriority',
 'value': 1000}


In [10]:
penaltyValue = priorityClassResource['value']

penaltyValue

1000

# Creating PriorityClass

## Body 정의

In [11]:
priority_body = {
    "apiVersion": "scheduling.k8s.io/v1",
    "kind": "PriorityClass",
    "metadata": {"name" : "high-priority"},
    "preemptionPolicy": "PreemptLowerPriority",
    "value" : 10001
}

## Create

In [14]:
created_resource = api.create_cluster_custom_object(
    group="scheduling.k8s.io",
    version="v1",
    plural="priorityclasses",
    body=priority_body
)

In [15]:
get_created_resource = api.get_cluster_custom_object(
    group="scheduling.k8s.io",
    version="v1",
    plural="priorityclasses",
    name="high-priority"
)

pprint.pprint(get_created_resource)

{'apiVersion': 'scheduling.k8s.io/v1',
 'kind': 'PriorityClass',
 'metadata': {'creationTimestamp': '2023-06-01T13:41:40Z',
              'generation': 1,
              'managedFields': [{'apiVersion': 'scheduling.k8s.io/v1',
                                 'fieldsType': 'FieldsV1',
                                 'fieldsV1': {'f:preemptionPolicy': {},
                                              'f:value': {}},
                                 'manager': 'OpenAPI-Generator',
                                 'operation': 'Update',
                                 'time': '2023-06-01T13:41:40Z'}],
              'name': 'high-priority',
              'resourceVersion': '530600',
              'uid': 'a2eb8747-1183-4e42-b7ec-a931f33e974d'},
 'preemptionPolicy': 'PreemptLowerPriority',
 'value': 10001}


## Delete

In [16]:
result = api.delete_cluster_custom_object(
    group="scheduling.k8s.io",
    version="v1",
    plural="priorityclasses",
    name="high-priority"
)

# PriorityClass Update 순서

### 1. PodGroup이 Pending에서 Running 상태가 된다.
### 2. 해당 PodGroup이 참조하는 PriorityClass를 찾아서 value값만 받아오고 삭제한다.
### 3. value에 penalty정책을 적용하여 갱신된 value 값을 같은 이름의 PriorityClass로 새로 만든다.
### 4. PodGroup의 상태가 Completed가 되면 count를 1개 올리고 penalty 정책에 반영한다.
### 5. 일정 시간동안 count의 변화가 없으면 PriorityClass의 value값을 점차 회복시킨다.